# Proyecto Final
## Procesamiento de Datos
##### Gonzalo Fernández Suárez

---

1) Crear un CALLER para descargar noticias de Cryptomonedas (2.0 puntos de 10)

1.1 - Conectarse a la API de twitters para descargar tweets sobre Cryptomonedas (0.5 puntos de 10).

1.2 - Conectarse a la API de Gdelt para descargar noticias sobre Cryptomonedas (0.5 puntos de 10).

1.3 - Scrapear la web https://coinmarketcal.com/en/news para descargar noticicas sobre Cryptomonedas (1.0 puntos de 10).

2) Crear un SERVER de Spark Streamming (1 puntos de 10)

3) Hacer un analisis de los mensajes en directo en clase, con ventana de tiempo (1 puntos de 10)

4) Aplicar un modelo de analisis de sentimiento (2.5 puntos de 10)

4.1 - Aplicar uno de los modelo como Vader Sentiment Analysis, textblob, flair o FinBERT (+0.5 puntos de 10).

4.2 - Aplicar todos los modelo: Vader Sentiment Analysis, textblob, flair y FinBERT (+0.5 puntos de 10).

4.3 - Crear un modelo de machine learning propio y aplicar dicho modelo a los mensajes (+1.5 puntos de 10).

Utilizando los siguientes datos etiquetados manualmente:
20220421_FULLDATASET_FOR_UTAD.csv

4.3.1 - Utilizar news como texto y final_manual_labelling como variable target para vuestro modelo!

 

5) Guardar los datos del SERVER en SQL marcados por tiempo (1.0 puntos de 10)

6) Presentar gráficas de evolución de los sentimiento y comparativa (1.5 puntos de 10)

6.1 - Presentar gráficas de evolución de los sentimiento, uno de los modelo como Vader Sentiment Analysis, textblob y flair (+1.0 puntos de 10).

6.2 - Presentar gráficas de evolución de los sentimiento de todos los modelo: Vader Sentiment Analysis, textblob y flair (+0.5 puntos de 10).

7) Incluir en la comparativa tu propio modelo, y ganas nota máxima si tu modelo es mejeor que los anteriores (+1.0 puntos de 10).

[Tutorial instalacion Spark en Windows](https://phoenixnap.com/kb/install-spark-on-windows-10).
El link de Java esta mal, descargar JDK no JRE

[Runear spark](https://aamargajbhiye.medium.com/apache-spark-setup-a-multi-node-standalone-cluster-on-windows-63d413296971)
```
cmd >> cd C:\Users\User\Desktop\U-Tad\Procesamiento de Datos\spark-3.2.1-bin-hadoop3.2

# Levantar Master Node
cmd >> bin\spark-class org.apache.spark.deploy.master.Master --host localhost
```
http://DESKTOP-GFS:8080

```
# Poner a escuchar al Worker Node
cmd >> bin\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost
```

# Aplicacion PYTHON

Nota: Este proyecto está desarrollado enteramente en un sistema **Windows**. Las máquinas virtuales Ubuntu me han dado malas experiencias petandome el ordenador o bien rompiendose dañando toda la informacion que habia en ellas.

## 1.3 Scrapear la web https://coinmarketcal.com/en/news para descargar noticicas sobre Cryptomonedas (1 PUNTO)

In [78]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [68]:
# Para mantener silenciado el webdriver_manager
os.environ['WDM_LOG'] = '0'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
driver.maximize_window()
wait = WebDriverWait(driver,10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\gonef\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [69]:
driver.get('https://coinmarketcal.com/en/news')

In [76]:
newsTitles = []
titles = driver.find_elements(By.CLASS_NAME, 'card__title')
for i in titles:
    print (i.text)
    newsTitles.append(i.text)

DEX protocol Uniswap hits $1 trillion cumulative trading volume milestone
New NFT Marketplace ‘Golom’ Loaded With Analytical Tools Announces Genesis Period
Merit Circle DAO proposal seeks to ‘trim the fat’ by booting early-stage investor
Tectona’s Bored.Ai Opens Text2Art Minting For The First Time
Adam Neumann-backed blockchain startup Flowcarbon raises $70 million
Optimism cracks down on airdrop farmers, removing 17,000 addresses
Coinbase becomes the first crypto company to enter the Fortune 500
HyperEarn Officially Opens Its Token IDO
Ted Cruz: ‘I want Texas to be the oasis on planet Earth for bitcoin and crypto’
SW DAO Monthly Report: April 2022 Edition


In [75]:
# Next Page
nextPageButton = driver.find_element(By.XPATH,'/html/body/main/section[1]/div[2]/div[2]/div/div/nav/ul/li[6]/a')
driver.execute_script("arguments[0].scrollIntoView();", nextPageButton)
nextPageButton.click()

In [81]:
df = pd.DataFrame()
df['news'] = newsTitles

In [82]:
df

,news
0,DEX protocol Uniswap hits $1 trillion cumulati...
1,New NFT Marketplace ‘Golom’ Loaded With Analyt...
2,Merit Circle DAO proposal seeks to ‘trim the f...
3,Tectona’s Bored.Ai Opens Text2Art Minting For ...
4,Adam Neumann-backed blockchain startup Flowcar...
5,"Optimism cracks down on airdrop farmers, remov..."
6,Coinbase becomes the first crypto company to e...
7,HyperEarn Officially Opens Its Token IDO
8,Ted Cruz: ‘I want Texas to be the oasis on pla...
9,SW DAO Monthly Report: April 2022 Edition


## 2 Crear un SERVER de Spark Streamming (1 PUNTO)

In [5]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [30]:
# Inicializamos el Contexto
sc = SparkContext("local[2]" , "NetworkWordCount" ) # Master y nombre de la app
ssc = StreamingContext(sc, 10) # Contexto de streaming, contexto de spark y cada cuanto se repite el proceso

# Inicializamos el socket => Comunicamos el origen de los datos
lines = ssc.socketTextStream("localhost", 9090) # Host, Puerto

## 3 Hacer un analisis de los mensajes en directo, con ventana de tiempo (1 PUNTO)

In [5]:
# Como vamos a procesar los datos haciendo duplas con las palabras recibidas y el numero de veces que se repiten
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word,1))
wordCounts = pairs.reduceByKey(lambda x,y: x+y)

wordCounts.pprint()

Ahora en la consola de comandos nos conectamos, usando netCat, a "localhost" en el puerto 9090. 

[Instalación netCat en Windows](https://programmerclick.com/article/5414801604/)

Abrimos la consola de comandos y ejecutamos:
```
cmd >> nc -L -p 9090    # Nos conectamos al escuchador del puerto 9090
```
Ahora desde la cmd podemos enviar información en tiempo real a Spark para que la analice.

In [6]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

-------------------------------------------
Time: 2022-05-01 18:23:20
-------------------------------------------
('Hola', 1)

-------------------------------------------
Time: 2022-05-01 18:23:30
-------------------------------------------
('Soy', 1)
('Gonzalo', 1)
('probando', 1)
('Estoy', 1)

-------------------------------------------
Time: 2022-05-01 18:23:40
-------------------------------------------
('como', 1)
('Spark', 1)
('funciona', 1)
('Streaming', 1)

-------------------------------------------
Time: 2022-05-01 18:23:50
-------------------------------------------
('mensajes', 1)
('en', 1)
('Con', 1)
('directo', 1)

-------------------------------------------
Time: 2022-05-01 18:24:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-01 18:24:10
-------------------------------------------

-------------------------------------------
Time: 2022-05-01 18:24:20
-------------------------------------------



In [ ]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()

## 4 Aplicar un modelo de analisis de sentimiento a los mensajes

### 4.1 Aplicar uno de los modelo como Vader Sentiment Analysis, textblob, flair o FinBERT (0,5 PUNTOS)

##### Vader Sentiment Analysis

```
>> pip install vaderSentiment
```

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
# Generamos un nuevo Server de Spark Streaming para el Vader Sentiment Analysis
sc = SparkContext("local[2]" , "NetworkVaderAnalysis" ) 
ssc = StreamingContext(sc, 10) 
lines = ssc.socketTextStream("localhost", 9090) 

In [3]:
# Procesamiento de los datos
vader = SentimentIntensityAnalyzer() # Inicializamos el anlazidador de Vader
# Genero Tuplas con la Linea recibida y su valor 'compound' del Vader Sentiment Analysis
linesAnalyzed = lines.map(lambda line: (line,vader.polarity_scores(line)['compound']))
# Pinto el resultado
linesAnalyzed.pprint()

```
cmd >> nc -L -p 9090    # Nos conectamos al escuchador del puerto 9090
```

In [4]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

-------------------------------------------
Time: 2022-05-03 13:50:50
-------------------------------------------
('hello, good morning', 0.4404)

-------------------------------------------
Time: 2022-05-03 13:51:00
-------------------------------------------
('my name is Gonzalo', 0.0)

-------------------------------------------
Time: 2022-05-03 13:51:10
-------------------------------------------
('im a software lover', 0.5859)

-------------------------------------------
Time: 2022-05-03 13:51:20
-------------------------------------------

-------------------------------------------
Time: 2022-05-03 13:51:30
-------------------------------------------
('and rigth now im testing spark streaming doing a sentiment analysis of my text', 0.2263)

-------------------------------------------
Time: 2022-05-03 13:51:40
-------------------------------------------

-------------------------------------------
Time: 2022-05-03 13:51:50
-------------------------------------------
('i hope this

In [ ]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()

### 4.2 Aplicar todos los modelo: Vader Sentiment Analysis, textblob, flair y FinBERT (0,5 PUNTOS)

Apliquemos al mismo tiempo todos los modelos de Sentiment Analysis a las lineas de texto.

```
>> pip install vaderSentiment
>> pip install textblob
>> pip install sentencepiece
>> pip install flair   
>> pip install transformers
```

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
# Vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# TextBlob
from textblob import TextBlob
# Flair
from flair.models import TextClassifier
from flair.data import Sentence
# FinBert
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [2]:
# Generamos un nuevo Server de Spark Streaming para el Vader Sentiment Analysis
# spark = SparkSession.builder.appName('Sentiment Analysis').master('local').getOrCreate()
sc = SparkContext("local[2]" , "NetworkVaderAnalysis" ) 
ssc = StreamingContext(sc, 10) 
lines = ssc.socketTextStream("localhost", 9090) 

In [3]:
# Prerequisitos
# Vader
vader = SentimentIntensityAnalyzer() # Inicializamos el anlazidador de Vader

# Flair
classifier = TextClassifier.load('en-sentiment') # Inicializamos el clasificador de Flair en ingles
def apply_fair(x):  # Definimos la funcion que aplicará el analisis de Flair a cada texto
    sentence = Sentence(x)
    classifier.predict(sentence)
    return sentence.labels[0].to_dict()['confidence']

# FinBert
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
def apply_finbert(x):
#    return (0,1,2)
    inputs = tokenizer([x], padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=1) 
    return predictions[:, 0].tolist()[0], predictions[:, 1].tolist()[0], predictions[:, 2].tolist()[0]


2022-05-05 12:37:43,756 loading file C:\Users\gonef\.flair\models\sentiment-en-mix-distillbert_4.pt


In [83]:
print(apply_fair('hello, today is a nice day')) # El analisis con Flair funciona
apply_finbert('hello, today is a nice day')[0] # El analisis con FinBert funciona

0.8923711180686951


0.7736659049987793

In [4]:
## Como vamos a procesar los datos
# Generamos tuplas con la linea analizada y el resultado obtenido de cada tecnica de Sentiment Analysis aplicada
vaderPair = lines.map(lambda line: (line, vader.polarity_scores(line)['compound']) ) # Vader
textBlobPair = lines.map(lambda line : (line, TextBlob(line).sentiment.polarity) ) # TextBlob
#flairPair = lines.map(lambda line: (line, apply_fair(line))) # Flair  Excepcion en .awaitTermination() y no hay forma de solucionarlo
#finBertPair = lines.map(lambda line: (line, apply_finbert(line)[0]) ) # FinBert  Excepcion en .awaitTermination() y no hay forma de solucionarlo

result = vaderPair.join(textBlobPair)#.join(flairPair).join(finBertPair)

result.pprint()

```
cmd >> nc -L -p 9090    # Nos conectamos al escuchador del puerto 9090
```

In [ ]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

In [106]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()

### 4.3 - Crear un modelo de machine learning propio (1,5 PUNTOS)

Utilizar news como texto y final_manual_labelling como variable target para vuestro modelo!

In [217]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="ProyectoFinal")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [218]:
df = spark.read.csv('Data/20220421_FULLDATASET_FOR_UTAD.csv',inferSchema=True,header=True)

In [219]:
print((df.count(),len(df.columns)))

(2683, 4)


In [220]:
df.show(5)

+---+---------+--------------------+----------------------+
| id|     date|                news|final_manual_labelling|
+---+---------+--------------------+----------------------+
|  0|1/25/2022|Ripple announces ...|                     1|
|  1|1/25/2022|IMF directors urg...|                    -1|
|  2|1/25/2022|Dragonfly Capital...|                     1|
|  3|1/25/2022|Rick and Morty co...|                     0|
|  4|1/25/2022|How fintech SPACs...|                     0|
+---+---------+--------------------+----------------------+
only showing top 5 rows



In [221]:
df = df.filter(df['final_manual_labelling'] != 0) # Elimino las noticias neutras

In [222]:
from pyspark.sql.functions import *
df = df.withColumn('final_manual_labelling',when(df.final_manual_labelling == -1,0).otherwise(1)) # Cambio los -1 por 0

In [223]:
df.show(10)

+---+---------+--------------------+----------------------+
| id|     date|                news|final_manual_labelling|
+---+---------+--------------------+----------------------+
|  0|1/25/2022|Ripple announces ...|                     1|
|  1|1/25/2022|IMF directors urg...|                     0|
|  2|1/25/2022|Dragonfly Capital...|                     1|
|  5|1/25/2022|Multichain vulner...|                     0|
|  8|1/25/2022|GoodDollar Launch...|                     1|
|  9|1/25/2022|BCB Group raises ...|                     1|
| 10|1/25/2022|Carola Morena to ...|                     1|
| 11|1/25/2022|Twitter is growin...|                     1|
| 12|1/24/2022|Bitcoin climbs mo...|                     1|
| 13|1/24/2022|Walmart director ...|                     1|
+---+---------+--------------------+----------------------+
only showing top 10 rows



In [224]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, date: string, news: string, final_manual_labelling: int]>

In [225]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer

In [226]:
# Indexing
search_engine_class = StringIndexer(inputCol="news", outputCol="news_num")
search_engine_obj = search_engine_class.fit(df)
df = search_engine_obj.transform(df)

In [227]:
df.show(5)

+---+---------+--------------------+----------------------+--------+
| id|     date|                news|final_manual_labelling|news_num|
+---+---------+--------------------+----------------------+--------+
|  0|1/25/2022|Ripple announces ...|                     1|  1127.0|
|  1|1/25/2022|IMF directors urg...|                     0|   789.0|
|  2|1/25/2022|Dragonfly Capital...|                     1|   569.0|
|  5|1/25/2022|Multichain vulner...|                     0|   955.0|
|  8|1/25/2022|GoodDollar Launch...|                     1|   742.0|
+---+---------+--------------------+----------------------+--------+
only showing top 5 rows



In [228]:
#one hot encoding
search_engine_encoder = OneHotEncoder(inputCol="news_num", outputCol="news_vector")
ohe = search_engine_encoder.fit(df)
df = ohe.transform(df)

In [229]:
df.show(5)

+---+---------+--------------------+----------------------+--------+-------------------+
| id|     date|                news|final_manual_labelling|news_num|        news_vector|
+---+---------+--------------------+----------------------+--------+-------------------+
|  0|1/25/2022|Ripple announces ...|                     1|  1127.0|(1441,[1127],[1.0])|
|  1|1/25/2022|IMF directors urg...|                     0|   789.0| (1441,[789],[1.0])|
|  2|1/25/2022|Dragonfly Capital...|                     1|   569.0| (1441,[569],[1.0])|
|  5|1/25/2022|Multichain vulner...|                     0|   955.0| (1441,[955],[1.0])|
|  8|1/25/2022|GoodDollar Launch...|                     1|   742.0| (1441,[742],[1.0])|
+---+---------+--------------------+----------------------+--------+-------------------+
only showing top 5 rows



In [230]:
from pyspark.ml.feature import VectorAssembler

In [231]:
df_assembler = VectorAssembler(inputCols=['news_vector'], outputCol="features")
df = df_assembler.transform(df)

In [232]:
#select data for building model
model_df=df.select(['features','final_manual_labelling'])

In [233]:
from pyspark.ml.classification import LogisticRegression

In [234]:
#split the data 
training_df,test_df=model_df.randomSplit([0.75,0.25])
(training_df.count(),test_df.count())

(1072, 370)

In [235]:
log_reg_class=LogisticRegression(labelCol='final_manual_labelling')
log_reg_model=log_reg_class.fit(training_df)

In [236]:
#Training Results
train_results=log_reg_model.evaluate(training_df).predictions

c:\Users\gonef\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [237]:
# Probability at 0 index is for 0 class and probabilty as 1 index is for 1 class
correct_preds=train_results.filter(train_results['final_manual_labelling']==1).filter(train_results['prediction']==1).count()
#accuracy on training dataset 
float(correct_preds)/(training_df.filter(training_df['final_manual_labelling']==1).count())

1.0

In [238]:
# Test Set results
results=log_reg_model.evaluate(test_df).predictions

In [239]:
results.select(['final_manual_labelling','prediction']).show(10,False)

+----------------------+----------+
|final_manual_labelling|prediction|
+----------------------+----------+
|1                     |1.0       |
|1                     |1.0       |
|1                     |1.0       |
|0                     |1.0       |
|1                     |1.0       |
|1                     |1.0       |
|1                     |1.0       |
|1                     |1.0       |
|1                     |1.0       |
|1                     |1.0       |
+----------------------+----------+
only showing top 10 rows



In [240]:
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- final_manual_labelling: integer (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [241]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
#confusion matrix
true_postives = results[(results.final_manual_labelling == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.final_manual_labelling == 0) & (results.prediction == 0)].count()
false_positives = results[(results.final_manual_labelling == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.final_manual_labelling == 1) & (results.prediction == 0)].count()

In [242]:
print (true_postives)
print (true_negatives)
print (false_positives)
print (false_negatives)
print(true_postives+true_negatives+false_positives+false_negatives)
print (results.count())

289
0
81
0
370
370


In [243]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

1.0


In [244]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.7810810810810811


In [245]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.7810810810810811


In [248]:
spark.stop()

### 5 Guardar los datos del SERVER en SQL marcados por tiempo (1 PUNTO)

In [249]:
from pyspark.sql import SQLContext 
from pyspark.sql.session import SparkSession

In [250]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="ProyectoFinal")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
sql = SQLContext(sc)

c:\Users\gonef\anaconda3\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [251]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("nullValue", "NA") \
    .option("mode", "dropMalformed") \
    .csv("Data/20220421_FULLDATASET_FOR_UTAD.csv")

In [252]:
df.show(10)

+---+---------+--------------------+----------------------+
| id|     date|                news|final_manual_labelling|
+---+---------+--------------------+----------------------+
|  0|1/25/2022|Ripple announces ...|                     1|
|  1|1/25/2022|IMF directors urg...|                    -1|
|  2|1/25/2022|Dragonfly Capital...|                     1|
|  3|1/25/2022|Rick and Morty co...|                     0|
|  4|1/25/2022|How fintech SPACs...|                     0|
|  5|1/25/2022|Multichain vulner...|                    -1|
|  6|1/25/2022|YouTube wants to ...|                     0|
|  7|1/25/2022|OpenSea is reimbu...|                     0|
|  8|1/25/2022|GoodDollar Launch...|                     1|
|  9|1/25/2022|BCB Group raises ...|                     1|
+---+---------+--------------------+----------------------+
only showing top 10 rows



In [255]:
#result.registerTemplate("results")

NameError: name 'result' is not defined

In [253]:
df.registerTempTable("fulldataset")

c:\Users\gonef\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [254]:
spark.stop()
sc.stop()